<a href="https://colab.research.google.com/github/EmircanTanyildiz/PythonHealthSignal/blob/main/EmirCanTanyildizHealthSignalCNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.models import Sequential, Model,load_model

from keras.preprocessing.image import load_img, img_to_array
import keras
print("kütüphaneler Yüklendi")

kütüphaneler Yüklendi


In [ ]:
import keras.preprocessing.image as ImageDataGenerator


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from keras.applications.mobilenet import MobileNet, preprocess_input

In [ ]:
import keras

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


In [ ]:
def preprocessingImage1(path):

  image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, rescale=1/255, horizontal_flip=True)
  image= image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size=32, class_mode='binary')

  return image


In [ ]:
def preprocessingImage2(path):
  image_data = ImageDataGenerator(rescale=1/255)
  image = image_data.flow_from_directory(directory = path, target_size = (224,224), batch_size = 32, class_mode='binary')
  return image


In [ ]:
path = "/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Train/"
train_data = preprocessingImage1(path)

Found 2905 images belonging to 2 classes.


In [ ]:
train_data.class_indices

{'Brain Tumor': 0, 'Healthy': 1}

In [ ]:
path = "/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/"
test_data = preprocessingImage2(path)

Found 620 images belonging to 2 classes.


In [ ]:
path = "/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Valid/"
valid_data = preprocessingImage2(path)

Found 641 images belonging to 2 classes.


In [ ]:
from PIL import Image

In [ ]:
from keras.preprocessing import image

In [ ]:
base_model = MobileNet(input_shape=(240,240,3), include_top=False)


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
X = Flatten()(base_model.output)
X = Dense(units= 1, activation='sigmoid')(X)
model = Model(base_model.input, X)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 120, 120, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 120, 120, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 120, 120, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 120, 120, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 120, 120, 32)      128 

In [ ]:
import keras

In [ ]:
model.compile(optimizer = 'rmsprop', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
#CallBack

mc = ModelCheckpoint(filepath='/content/drive/MyDrive/ENGPROJECT3/archive/bestmodel4.keras', monitor='val_accuracy', verbose=1, save_best_only=True)
#Early Stopping
es =  EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=3, verbose=1)
cb = [mc,es]

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
model = load_model("/content/drive/MyDrive/ENGPROJECT3/archive/bestmodel.keras")

In [ ]:
acc = model.evaluate(test_data)[1]
print(f"Modelin Dogruluk Orani: %{acc*100}")

20/20 [==============================] - 28s 1s/step - loss: 0.4205 - accuracy: 0.9516
Modelin Dogruluk Orani: %95.16128897666931


In [ ]:
def tahminEt(_Path):
    path = _Path
    img = image.load_img(path, target_size=(224,224))
    i = image.img_to_array(img)/255
    input_arr = np.array([i])
    input_arr.shape

    pred = np.argmax(model.predict(input_arr))

    if pred == 0:
        print("MRI Görüntüsünde Beyin Tümörüne Rastlanıldı")
    else:
        print("MRI Görüntürüsünde Beyin Tümörüne Rastlanılmadı")
    plt.imshow(input_arr[0])
    plt.title("input image")
    plt.show()


In [ ]:
def tahminEt(_Path):
    path = _Path
    img = image.load_img(path, target_size=(224,224))
    i = image.img_to_array(img)
    i = preprocess_input(i)
    input_arr = np.array([i])
    input_arr.shape

    pred = model.predict(input_arr)[0][0]  # Sigmoid aktivasyonundan dolayı 0 ile 1 arasında bir değer döner

    if pred > 0.5:
        print("MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X")
        return 0
    else:
        print("MRI Görüntüsünde Beyin Tümörüne Rastlanıldı Y")
        return 1



In [ ]:
tahminEt("/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/Healthy/Not Cancer  (1003).jpg")

1/1 [==============================] - 1s 802ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0

In [ ]:
tahminEt("/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/Healthy/Not Cancer  (1103).jpg")

1/1 [==============================] - 0s 88ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0

In [ ]:
tahminEt("/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/Brain Tumor/Cancer (1).jpg")

1/1 [==============================] - 0s 59ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanıldı Y


1

In [ ]:
tahminEt("/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/Healthy/Not Cancer  (1840).jpg")

1/1 [==============================] - 0s 57ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0

In [ ]:
tahminEt("/content/drive/MyDrive/ENGPROJECT3/archive/BrainTumorDataSet/BrainTumorDataSet/Test/Healthy/Not Cancer  (436).jpg")

1/1 [==============================] - 0s 54ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0

In [ ]:
tahminEt("/content/drive/MyDrive/archive/no/N_1001.jpg")

1/1 [==============================] - 0s 54ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0

In [ ]:
tahminEt("/content/drive/MyDrive/archive/yes/Y_1011.jpg")

1/1 [==============================] - 0s 54ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanıldı Y


1

In [ ]:
tahminEt("/content/drive/MyDrive/archive/no/N_1043.jpg")

1/1 [==============================] - 0s 55ms/step
MRI Görüntüsünde Beyin Tümörüne Rastlanılmadı X


0